In [1]:
import pandas as pd

df = pd.read_csv("../data/cleaned_premier_league.csv")
print(df.shape)
print(df.columns)
print(df.head)

(9500, 26)
Index(['MatchID', 'Season', 'MatchWeek', 'Date', 'HomeTeam', 'AwayTeam',
       'FullTimeHomeTeamGoals', 'FullTimeAwayTeamGoals', 'FullTimeResult',
       'HalfTimeHomeTeamGoals', 'HalfTimeAwayTeamGoals', 'HalfTimeResult',
       'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsOnTarget',
       'AwayTeamShotsOnTarget', 'HomeTeamCorners', 'AwayTeamCorners',
       'HomeTeamFouls', 'AwayTeamFouls', 'HomeTeamYellowCards',
       'AwayTeamYellowCards', 'HomeTeamRedCards', 'AwayTeamRedCards',
       'HomeTeamPoints', 'AwayTeamPoints'],
      dtype='object')
<bound method NDFrame.head of                                MatchID     Season  MatchWeek        Date  \
0          2000-2001_Charlton_Man City  2000-2001          1  2000-08-19   
1           2000-2001_Chelsea_West Ham  2000-2001          1  2000-08-19   
2     2000-2001_Coventry_Middlesbrough  2000-2001          1  2000-08-19   
3          2000-2001_Derby_Southampton  2000-2001          1  2000-08-19   
4              2000-

In [3]:
# rolling average goal scoring form for both Home and Away
df = df.sort_values("Date").reset_index(drop=True)

df["HomeGoalScoringForm"] = (
    df.groupby("HomeTeam")["FullTimeHomeTeamGoals"]
    .rolling(window=5, min_periods=1)
    .mean()
    .shift(1)
    .reset_index(level=0, drop=True)
)

df["AwayGoalScoringForm"] = (
    df.groupby("AwayTeam")["FullTimeAwayTeamGoals"]
    .rolling(window=5, min_periods=1)
    .mean()
    .shift(1)
    .reset_index(level=0, drop=True)
)

print(df["HomeGoalScoringForm"])
print(df["AwayGoalScoringForm"])

0       1.0
1       0.8
2       1.6
3       1.6
4       0.8
       ... 
9495    0.8
9496    0.6
9497    3.2
9498    0.8
9499    1.8
Name: HomeGoalScoringForm, Length: 9500, dtype: float64
0       0.8
1       1.8
2       1.0
3       0.8
4       0.6
       ... 
9495    0.6
9496    1.6
9497    1.0
9498    1.8
9499    1.4
Name: AwayGoalScoringForm, Length: 9500, dtype: float64
